In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%load_ext autoreload
%autoreload 2

import pandas as pd
pd.options.display.max_columns=1000 
pd.options.display.max_rows=1000
import numpy as np

In [2]:
import sys
sys.path.append('ml_modules/')

In [3]:
DATA_DIR = ''

train = pd.read_csv(DATA_DIR + 'train.csv')
"train", train.shape

test = pd.read_csv(DATA_DIR + 'test_QyjYwdj.csv')
"test", test.shape

campaign_data = pd.read_csv(DATA_DIR + 'campaign_data.csv')
campaign_data['start_date'] = pd.to_datetime(campaign_data.start_date, format='%d/%m/%y')
campaign_data['end_date'] = pd.to_datetime(campaign_data.end_date,  format='%d/%m/%y')
campaign_data['duration'] = (campaign_data.end_date - campaign_data.start_date).dt.days

('train', (78369, 5))

('test', (50226, 4))

Merge Features

In [4]:
cust_hist_trans = pd.read_csv('cust_hist_trans.csv')
cust_hist_trans['start_date'] = pd.to_datetime(cust_hist_trans.start_date)

cust_hist_trans_daily = pd.read_csv('cust_hist_trans_daily.csv')
cust_hist_trans_daily['start_date'] = pd.to_datetime(cust_hist_trans_daily.start_date)

coup_hist_trans = pd.read_csv('coup_hist_trans.csv')
coup_hist_trans['start_date'] = pd.to_datetime(coup_hist_trans.start_date)

coup_hist_trans_daily = pd.read_csv('coup_hist_trans_daily.csv')
coup_hist_trans_daily['start_date'] = pd.to_datetime(coup_hist_trans_daily.start_date)

cust_coup_hist_trans = pd.read_csv('cust_coup_hist_trans.csv')
cust_coup_hist_trans['start_date'] = pd.to_datetime(cust_coup_hist_trans.start_date)

cust_coup_hist_trans_daily = pd.read_csv('cust_coup_hist_trans_daily.csv')
cust_coup_hist_trans_daily['start_date'] = pd.to_datetime(cust_coup_hist_trans_daily.start_date)

coup_brand_hist_trans = pd.read_csv('coup_brand_hist_trans.csv')
coup_brand_hist_trans['start_date'] = pd.to_datetime(coup_brand_hist_trans.start_date)

cust_brand_hist_trans = pd.read_csv('cust_brand_hist_trans.csv')
cust_brand_hist_trans['start_date'] = pd.to_datetime(cust_brand_hist_trans.start_date)

cust_coup_brand_hist_trans = pd.read_csv('cust_coup_brand_hist_trans.csv')
cust_coup_brand_hist_trans['start_date'] = pd.to_datetime(cust_coup_brand_hist_trans.start_date)

In [5]:
def merge_dfs(df):
    df = df.merge(campaign_data, on='campaign_id', how='left', suffixes=['', '_camp']
                  
    ).merge(coup_hist_trans, on=['coupon_id', 'start_date'], how='left', suffixes=['', '_coht']
    ).merge(coup_hist_trans_daily, on=['coupon_id', 'start_date'], how='left', suffixes=['', '_cohtd']
            
    ).merge(cust_hist_trans, on=['customer_id', 'start_date'], how='left', suffixes=['', '_cuht']
    ).merge(cust_hist_trans_daily, on=['customer_id', 'start_date'], how='left', suffixes=['', '_cuhtd']
            
    ).merge(cust_coup_hist_trans, on=['customer_id', 'coupon_id','start_date'], how='left', suffixes=['', '_cucoht']
    ).merge(cust_coup_hist_trans_daily, on=['customer_id', 'coupon_id','start_date'], how='left', suffixes=['', '_cucohtd']
            
    ).merge(coup_brand_hist_trans, on=['coupon_id', 'start_date'], how='left', suffixes=['', '_cobrht']
    ).merge(cust_brand_hist_trans, on=['customer_id', 'start_date'], how='left', suffixes=['', '_cubrht']
    ).merge(cust_coup_brand_hist_trans, on=['customer_id', 'coupon_id','start_date'], how='left', suffixes=['', '_cucobrht']
            
    )
    df.shape
    df.head()
    return df
train.shape, test.shape
train = merge_dfs(train)
test = merge_dfs(test)

train.shape, test.shape

((78369, 5), (50226, 4))

((78369, 126), (50226, 125))

In [6]:
train.columns.tolist()

['id',
 'campaign_id',
 'coupon_id',
 'customer_id',
 'redemption_status',
 'campaign_type',
 'start_date',
 'end_date',
 'duration',
 'item_id_count',
 'selling_price_sum',
 'selling_price_mean',
 'selling_price_std',
 'other_discount_sum',
 'other_discount_mean',
 'coupon_discount_sum',
 'coupon_discount_mean',
 'coupon_applied_sum',
 'coupon_applied_mean',
 'date_count',
 'selling_price_mean_cohtd',
 'selling_price_std_cohtd',
 'selling_price_last',
 'other_discount_mean_cohtd',
 'other_discount_std',
 'other_discount_last',
 'coupon_discount_mean_cohtd',
 'coupon_discount_std',
 'coupon_discount_last',
 'coupon_applied_mean_cohtd',
 'coupon_applied_std',
 'coupon_applied_last',
 'selling_price_ewm_mean',
 'selling_price_ewm_std',
 'coupon_discount_ewm_mean',
 'coupon_discount_ewm_std',
 'coupon_applied_ewm_mean',
 'coupon_applied_ewm_std',
 'item_id_count_cuht',
 'selling_price_sum_cuht',
 'selling_price_mean_cuht',
 'selling_price_std_cuht',
 'other_discount_sum_cuht',
 'other_dis

In [7]:
id_col = 'id'
target_col = 'redemption_status'

columns_to_drop = []
columns_to_drop = [c for c in columns_to_drop if c in train.columns]

cat_cols = [
    'campaign_id',
    'coupon_id',
    'customer_id',
    'campaign_type',
    'start_date',
    'end_date',    
]
cat_cols = [c for c in cat_cols if c in train.columns]
cat_cols

['campaign_id',
 'coupon_id',
 'customer_id',
 'campaign_type',
 'start_date',
 'end_date']

In [8]:
# imputing categorical columns
train[cat_cols] = train[cat_cols].fillna('Missing')
test[cat_cols] = test[cat_cols].fillna('Missing')

# imputing numerical columns
train = train.fillna(-1)
test = test.fillna(-1)

In [9]:
group_col = train['campaign_id'].astype(str) 
group_col_coupon = train['coupon_id'].astype(str)
train.shape
train.head()

(78369, 126)

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,duration,item_id_count,selling_price_sum,selling_price_mean,selling_price_std,other_discount_sum,other_discount_mean,coupon_discount_sum,coupon_discount_mean,coupon_applied_sum,coupon_applied_mean,date_count,selling_price_mean_cohtd,selling_price_std_cohtd,selling_price_last,other_discount_mean_cohtd,other_discount_std,other_discount_last,coupon_discount_mean_cohtd,coupon_discount_std,coupon_discount_last,coupon_applied_mean_cohtd,coupon_applied_std,coupon_applied_last,selling_price_ewm_mean,selling_price_ewm_std,coupon_discount_ewm_mean,coupon_discount_ewm_std,coupon_applied_ewm_mean,coupon_applied_ewm_std,item_id_count_cuht,selling_price_sum_cuht,selling_price_mean_cuht,selling_price_std_cuht,other_discount_sum_cuht,other_discount_mean_cuht,coupon_discount_sum_cuht,coupon_discount_mean_cuht,coupon_applied_sum_cuht,coupon_applied_mean_cuht,date_count_cuhtd,selling_price_mean_cuhtd,selling_price_std_cuhtd,selling_price_last_cuhtd,other_discount_mean_cuhtd,other_discount_std_cuhtd,other_discount_last_cuhtd,coupon_discount_mean_cuhtd,coupon_discount_std_cuhtd,coupon_discount_last_cuhtd,coupon_applied_mean_cuhtd,coupon_applied_std_cuhtd,coupon_applied_last_cuhtd,selling_price_ewm_mean_cuhtd,selling_price_ewm_std_cuhtd,coupon_discount_ewm_mean_cuhtd,coupon_discount_ewm_std_cuhtd,coupon_applied_ewm_mean_cuhtd,coupon_applied_ewm_std_cuhtd,item_id_count_cucoht,selling_price_sum_cucoht,selling_price_mean_cucoht,selling_price_std_cucoht,other_discount_sum_cucoht,other_discount_mean_cucoht,coupon_discount_sum_cucoht,coupon_discount_mean_cucoht,coupon_applied_sum_cucoht,coupon_applied_mean_cucoht,date_count_cucohtd,selling_price_mean_cucohtd,selling_price_std_cucohtd,selling_price_last_cucohtd,other_discount_mean_cucohtd,other_discount_std_cucohtd,other_discount_last_cucohtd,coupon_discount_mean_cucohtd,coupon_discount_std_cucohtd,coupon_discount_last_cucohtd,coupon_applied_mean_cucohtd,coupon_applied_std_cucohtd,coupon_applied_last_cucohtd,selling_price_ewm_mean_cucohtd,selling_price_ewm_std_cucohtd,coupon_discount_ewm_mean_cucohtd,coupon_discount_ewm_std_cucohtd,coupon_applied_ewm_mean_cucohtd,coupon_applied_ewm_std_cucohtd,item_id_count_cobrht,selling_price_sum_cobrht,selling_price_mean_cobrht,selling_price_std_cobrht,other_discount_sum_cobrht,other_discount_mean_cobrht,coupon_discount_sum_cobrht,coupon_discount_mean_cobrht,coupon_applied_sum_cobrht,coupon_applied_mean_cobrht,item_id_count_cubrht,selling_price_sum_cubrht,selling_price_mean_cubrht,selling_price_std_cubrht,other_discount_sum_cubrht,other_discount_mean_cubrht,coupon_discount_sum_cubrht,coupon_discount_mean_cubrht,coupon_applied_sum_cubrht,coupon_applied_mean_cubrht,item_id_count_cucobrht,selling_price_sum_cucobrht,selling_price_mean_cucobrht,selling_price_std_cucobrht,other_discount_sum_cucobrht,other_discount_mean_cucobrht,coupon_discount_sum_cucobrht,coupon_discount_mean_cucobrht,coupon_applied_sum_cucobrht,coupon_applied_mean_cucobrht
0,1,13,27,1053,0,X,2013-05-19,2013-07-05,47,3309.0,286278.21,86.515023,47.882148,207715.95,62.773028,4662.46,1.409024,158.0,0.047749,419.0,683.241551,539.584693,283.54,495.742124,561.417296,141.4,11.127589,37.025067,0.00,0.377088,0.795351,0.0,682.819199,404.264352,11.127188,26.336349,0.377079,0.587860,270,45754.09,169.459593,405.939171,7971.00,29.522222,0.00,0.000000,0,0.000000,73,626.768356,785.646696,362.61,109.191781,199.165691,43.46,0.000000,0.000000,0.0,0.000000,0.000000,0,626.095294,575.776022,0.000000,0.000000,0.000000,0.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,901.0,82671.50,91.755272,50.278461,51127.14,56.744883,2739.04,3.040000,70.0,0.077691,1662.0,217463.13,130.844242,177.121409,40888.55,24.602016,0.00,0.000000,0.0,0.000000,-1.0,-1.00,-1.000000,-1.000000,-1.00,-1.00,-1.0,-1.0,-1.0,-1.0
1,2,13,116,48,0,X,2013-05-19,2013-07-05,47,6

Importing Dependencies

In [10]:
from custom_estimator import Estimator
from encoding import FreqeuncyEncoding, LabelEncoding
from custom_fold_generator import FoldScheme
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

In [11]:
fE = FreqeuncyEncoding(categorical_columns=cat_cols, return_df=True)
# lE = LabelEncoding(categorical_columns=label_cat_cols, return_df=True)

In [12]:
train = fE.fit_transform(train)
test = fE.transform(test)

In [13]:
train.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,duration,item_id_count,selling_price_sum,selling_price_mean,selling_price_std,other_discount_sum,other_discount_mean,coupon_discount_sum,coupon_discount_mean,coupon_applied_sum,coupon_applied_mean,date_count,selling_price_mean_cohtd,selling_price_std_cohtd,selling_price_last,other_discount_mean_cohtd,other_discount_std,other_discount_last,coupon_discount_mean_cohtd,coupon_discount_std,coupon_discount_last,coupon_applied_mean_cohtd,coupon_applied_std,coupon_applied_last,selling_price_ewm_mean,selling_price_ewm_std,coupon_discount_ewm_mean,coupon_discount_ewm_std,coupon_applied_ewm_mean,coupon_applied_ewm_std,item_id_count_cuht,selling_price_sum_cuht,selling_price_mean_cuht,selling_price_std_cuht,other_discount_sum_cuht,other_discount_mean_cuht,coupon_discount_sum_cuht,coupon_discount_mean_cuht,coupon_applied_sum_cuht,coupon_applied_mean_cuht,date_count_cuhtd,selling_price_mean_cuhtd,selling_price_std_cuhtd,selling_price_last_cuhtd,other_discount_mean_cuhtd,other_discount_std_cuhtd,other_discount_last_cuhtd,coupon_discount_mean_cuhtd,coupon_discount_std_cuhtd,coupon_discount_last_cuhtd,coupon_applied_mean_cuhtd,coupon_applied_std_cuhtd,coupon_applied_last_cuhtd,selling_price_ewm_mean_cuhtd,selling_price_ewm_std_cuhtd,coupon_discount_ewm_mean_cuhtd,coupon_discount_ewm_std_cuhtd,coupon_applied_ewm_mean_cuhtd,coupon_applied_ewm_std_cuhtd,item_id_count_cucoht,selling_price_sum_cucoht,selling_price_mean_cucoht,selling_price_std_cucoht,other_discount_sum_cucoht,other_discount_mean_cucoht,coupon_discount_sum_cucoht,coupon_discount_mean_cucoht,coupon_applied_sum_cucoht,coupon_applied_mean_cucoht,date_count_cucohtd,selling_price_mean_cucohtd,selling_price_std_cucohtd,selling_price_last_cucohtd,other_discount_mean_cucohtd,other_discount_std_cucohtd,other_discount_last_cucohtd,coupon_discount_mean_cucohtd,coupon_discount_std_cucohtd,coupon_discount_last_cucohtd,coupon_applied_mean_cucohtd,coupon_applied_std_cucohtd,coupon_applied_last_cucohtd,selling_price_ewm_mean_cucohtd,selling_price_ewm_std_cucohtd,coupon_discount_ewm_mean_cucohtd,coupon_discount_ewm_std_cucohtd,coupon_applied_ewm_mean_cucohtd,coupon_applied_ewm_std_cucohtd,item_id_count_cobrht,selling_price_sum_cobrht,selling_price_mean_cobrht,selling_price_std_cobrht,other_discount_sum_cobrht,other_discount_mean_cobrht,coupon_discount_sum_cobrht,coupon_discount_mean_cobrht,coupon_applied_sum_cobrht,coupon_applied_mean_cobrht,item_id_count_cubrht,selling_price_sum_cubrht,selling_price_mean_cubrht,selling_price_std_cubrht,other_discount_sum_cubrht,other_discount_mean_cubrht,coupon_discount_sum_cubrht,coupon_discount_mean_cubrht,coupon_applied_sum_cubrht,coupon_applied_mean_cubrht,item_id_count_cucobrht,selling_price_sum_cucobrht,selling_price_mean_cucobrht,selling_price_std_cucobrht,other_discount_sum_cucobrht,other_discount_mean_cucobrht,coupon_discount_sum_cucobrht,coupon_discount_mean_cucobrht,coupon_applied_sum_cucobrht,coupon_applied_mean_cucobrht
0,1,0.288456,0.001557,0.000383,0,0.736056,0.288456,0.288456,47,3309.0,286278.21,86.515023,47.882148,207715.95,62.773028,4662.46,1.409024,158.0,0.047749,419.0,683.241551,539.584693,283.54,495.742124,561.417296,141.4,11.127589,37.025067,0.00,0.377088,0.795351,0.0,682.819199,404.264352,11.127188,26.336349,0.377079,0.587860,270,45754.09,169.459593,405.939171,7971.00,29.522222,0.00,0.000000,0,0.000000,73,626.768356,785.646696,362.61,109.191781,199.165691,43.46,0.000000,0.000000,0.0,0.000000,0.000000,0,626.095294,575.776022,0.000000,0.000000,0.000000,0.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,901.0,82671.50,91.755272,50.278461,51127.14,56.744883,2739.04,3.040000,70.0,0.077691,1662.0,217463.13,130.844242,177.121409,40888.55,24.602016,0.00,0.000000,0.0,0.000000,-1.0,-1.00,-1.000000,-1.000000,-1.00,-1.00,-1.0,-1.0,-1.0,-1.0
1,2,0.288456,0.001340,0.0

In [14]:
train.dtypes

id                                    int64
campaign_id                         float64
coupon_id                           float64
customer_id                         float64
redemption_status                     int64
campaign_type                       float64
start_date                          float64
end_date                            float64
duration                              int64
item_id_count                       float64
selling_price_sum                   float64
selling_price_mean                  float64
selling_price_std                   float64
other_discount_sum                  float64
other_discount_mean                 float64
coupon_discount_sum                 float64
coupon_discount_mean                float64
coupon_applied_sum                  float64
coupon_applied_mean                 float64
date_count                          float64
selling_price_mean_cohtd            float64
selling_price_std_cohtd             float64
selling_price_last              

In [15]:
test_ids = test[id_col]
train_ids = train[id_col]

In [16]:
y = train[target_col]
train.drop(columns=[x for x in [id_col] + columns_to_drop + [target_col] if x in train.columns], inplace=True)
test.drop(columns=[x for x in [id_col] + columns_to_drop + [target_col] if x in test.columns], inplace=True)

In [17]:
train.shape
train.head()

(78369, 124)

,campaign_id,coupon_id,customer_id,campaign_type,start_date,end_date,duration,item_id_count,selling_price_sum,selling_price_mean,selling_price_std,other_discount_sum,other_discount_mean,coupon_discount_sum,coupon_discount_mean,coupon_applied_sum,coupon_applied_mean,date_count,selling_price_mean_cohtd,selling_price_std_cohtd,selling_price_last,other_discount_mean_cohtd,other_discount_std,other_discount_last,coupon_discount_mean_cohtd,coupon_discount_std,coupon_discount_last,coupon_applied_mean_cohtd,coupon_applied_std,coupon_applied_last,selling_price_ewm_mean,selling_price_ewm_std,coupon_discount_ewm_mean,coupon_discount_ewm_std,coupon_applied_ewm_mean,coupon_applied_ewm_std,item_id_count_cuht,selling_price_sum_cuht,selling_price_mean_cuht,selling_price_std_cuht,other_discount_sum_cuht,other_discount_mean_cuht,coupon_discount_sum_cuht,coupon_discount_mean_cuht,coupon_applied_sum_cuht,coupon_applied_mean_cuht,date_count_cuhtd,selling_price_mean_cuhtd,selling_price_std_cuhtd,selling_price_last_cuhtd,other_discount_mean_cuhtd,other_discount_std_cuhtd,other_discount_last_cuhtd,coupon_discount_mean_cuhtd,coupon_discount_std_cuhtd,coupon_discount_last_cuhtd,coupon_applied_mean_cuhtd,coupon_applied_std_cuhtd,coupon_applied_last_cuhtd,selling_price_ewm_mean_cuhtd,selling_price_ewm_std_cuhtd,coupon_discount_ewm_mean_cuhtd,coupon_discount_ewm_std_cuhtd,coupon_applied_ewm_mean_cuhtd,coupon_applied_ewm_std_cuhtd,item_id_count_cucoht,selling_price_sum_cucoht,selling_price_mean_cucoht,selling_price_std_cucoht,other_discount_sum_cucoht,other_discount_mean_cucoht,coupon_discount_sum_cucoht,coupon_discount_mean_cucoht,coupon_applied_sum_cucoht,coupon_applied_mean_cucoht,date_count_cucohtd,selling_price_mean_cucohtd,selling_price_std_cucohtd,selling_price_last_cucohtd,other_discount_mean_cucohtd,other_discount_std_cucohtd,other_discount_last_cucohtd,coupon_discount_mean_cucohtd,coupon_discount_std_cucohtd,coupon_discount_last_cucohtd,coupon_applied_mean_cucohtd,coupon_applied_std_cucohtd,coupon_applied_last_cucohtd,selling_price_ewm_mean_cucohtd,selling_price_ewm_std_cucohtd,coupon_discount_ewm_mean_cucohtd,coupon_discount_ewm_std_cucohtd,coupon_applied_ewm_mean_cucohtd,coupon_applied_ewm_std_cucohtd,item_id_count_cobrht,selling_price_sum_cobrht,selling_price_mean_cobrht,selling_price_std_cobrht,other_discount_sum_cobrht,other_discount_mean_cobrht,coupon_discount_sum_cobrht,coupon_discount_mean_cobrht,coupon_applied_sum_cobrht,coupon_applied_mean_cobrht,item_id_count_cubrht,selling_price_sum_cubrht,selling_price_mean_cubrht,selling_price_std_cubrht,other_discount_sum_cubrht,other_discount_mean_cubrht,coupon_discount_sum_cubrht,coupon_discount_mean_cubrht,coupon_applied_sum_cubrht,coupon_applied_mean_cubrht,item_id_count_cucobrht,selling_price_sum_cucobrht,selling_price_mean_cucobrht,selling_price_std_cucobrht,other_discount_sum_cucobrht,other_discount_mean_cucobrht,coupon_discount_sum_cucobrht,coupon_discount_mean_cucobrht,coupon_applied_sum_cucobrht,coupon_applied_mean_cucobrht
0,0.288456,0.001557,0.000383,0.736056,0.288456,0.288456,47,3309.0,286278.21,86.515023,47.882148,207715.95,62.773028,4662.46,1.409024,158.0,0.047749,419.0,683.241551,539.584693,283.54,495.742124,561.417296,141.4,11.127589,37.025067,0.00,0.377088,0.795351,0.0,682.819199,404.264352,11.127188,26.336349,0.377079,0.587860,270,45754.09,169.459593,405.939171,7971.00,29.522222,0.00,0.000000,0,0.000000,73,626.768356,785.646696,362.61,109.191781,199.165691,43.46,0.000000,0.000000,0.0,0.000000,0.000000,0,626.095294,575.776022,0.000000,0.000000,0.000000,0.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,901.0,82671.50,91.755272,50.278461,51127.14,56.744883,2739.04,3.040000,70.0,0.077691,1662.0,217463.13,130.844242,177.121409,40888.55,24.602016,0.00,0.000000,0.0,0.000000,-1.0,-1.00,-1.000000,-1.000000,-1.00,-1.00,-1.0,-1.0,-1.0,-1.0
1,0.288456,0.001340,0.000191,0.736056,0.288456,0.2

LightGBM GroupKFold on CampaignID

In [18]:
params={
    'n_estimators': 20000, 
    'learning_rate': 0.1,
    'boosting_type': 'gbdt', 
    'colsample_bytree': 0.80,        
    'min_child_weight': 40.0,
    'num_leaves': 138, 
    'objective': 'binary', 
    'subsample': 0.50, 
    'subsample_freq': 5,
    'metric': 'custom'
}

lgb = Estimator(
    LGBMClassifier(**params), early_stopping_rounds=500, n_splits=5, random_state=100, 
    variance_penalty=1, verbose=100, eval_metric='AUC', scoring_metric=roc_auc_score, 
    validation_scheme=FoldScheme.GroupKFold, cv_group_col=group_col
)

lgb_oof = lgb.fit_transform(train.values, y.values)
lgb.avg_cv_score

Training until validation scores don't improve for 500 rounds
[100]	valid_0's auc: 0.939618
[200]	valid_0's auc: 0.938848
[300]	valid_0's auc: 0.939478
[400]	valid_0's auc: 0.935042
[500]	valid_0's auc: 0.936093
[600]	valid_0's auc: 0.937302
Early stopping, best iteration is:
[110]	valid_0's auc: 0.941149
Training until validation scores don't improve for 500 rounds
[100]	valid_0's auc: 0.952059
[200]	valid_0's auc: 0.953259
[300]	valid_0's auc: 0.952182
[400]	valid_0's auc: 0.952147
[500]	valid_0's auc: 0.949833
Early stopping, best iteration is:
[36]	valid_0's auc: 0.954712
Training until validation scores don't improve for 500 rounds
[100]	valid_0's auc: 0.847096
[200]	valid_0's auc: 0.847297
[300]	valid_0's auc: 0.846469
[400]	valid_0's auc: 0.837548
[500]	valid_0's auc: 0.833994
Early stopping, best iteration is:
[36]	valid_0's auc: 0.85379
Training until validation scores don't improve for 500 rounds
[100]	valid_0's auc: 0.873164
[200]	valid_0's auc: 0.881967
[300]	valid_0's auc:

0.9051266509809164

In [19]:
lgb_preds = lgb.transform(test[train.columns].values)

/Users/Arvind/Downloads/train_AUpWtIz/custom_estimator.py:188: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.mean(np.column_stack((est.predict_proba(x)[:,1] for est in self.fitted_models)), axis=1)


In [20]:
lgb.save_model(file_name='lgb-124-9130-gkf-camp.pkl')
pd.DataFrame({"id": train_ids, "redemption_status": lgb_oof}).to_csv('lgb-124-9130-gkf-camp-oof.csv', index=False)
pd.DataFrame({"id": test_ids, "redemption_status": lgb_preds}).to_csv('lgb-124-9130-gkf-camp-test.csv', index=False)

'lgb-124-9130-gkf-camp.pkl'

LightGBM GroupKFold on CouponId

In [21]:
params={
    'n_estimators': 20000, 
    'learning_rate': 0.01,
    'boosting_type': 'gbdt', 
    'colsample_bytree': 0.80,        
    'min_child_weight': 40.0,
    'num_leaves': 138, 
    'objective': 'binary', 
    'subsample': 0.50, 
    'subsample_freq': 5,
    'metric': 'custom'
}

lgb = Estimator(
    LGBMClassifier(**params), early_stopping_rounds=500, n_splits=5, random_state=100, 
    variance_penalty=1, verbose=100, eval_metric='AUC', scoring_metric=roc_auc_score, 
    validation_scheme=FoldScheme.GroupKFold, cv_group_col=group_col_coupon
)

lgb_oof = lgb.fit_transform(train.values, y.values)
lgb.avg_cv_score

Training until validation scores don't improve for 500 rounds
[100]	valid_0's auc: 0.924596
[200]	valid_0's auc: 0.937448
[300]	valid_0's auc: 0.940913
[400]	valid_0's auc: 0.941639
[500]	valid_0's auc: 0.942198
[600]	valid_0's auc: 0.943641
[700]	valid_0's auc: 0.943619
[800]	valid_0's auc: 0.945076
[900]	valid_0's auc: 0.945888
[1000]	valid_0's auc: 0.945735
[1100]	valid_0's auc: 0.946013
[1200]	valid_0's auc: 0.945962
[1300]	valid_0's auc: 0.946419
[1400]	valid_0's auc: 0.947446
[1500]	valid_0's auc: 0.948159
[1600]	valid_0's auc: 0.947749
[1700]	valid_0's auc: 0.948271
[1800]	valid_0's auc: 0.948111
[1900]	valid_0's auc: 0.947811
[2000]	valid_0's auc: 0.948265
[2100]	valid_0's auc: 0.948544
[2200]	valid_0's auc: 0.948785
[2300]	valid_0's auc: 0.949316
[2400]	valid_0's auc: 0.948835
[2500]	valid_0's auc: 0.948395
[2600]	valid_0's auc: 0.948866
[2700]	valid_0's auc: 0.949007
[2800]	valid_0's auc: 0.949078
Early stopping, best iteration is:
[2310]	valid_0's auc: 0.949503
Training unti

0.9405630446338609

In [22]:
lgb_preds = lgb.transform(test[train.columns].values)

/Users/Arvind/Downloads/train_AUpWtIz/custom_estimator.py:188: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.mean(np.column_stack((est.predict_proba(x)[:,1] for est in self.fitted_models)), axis=1)


In [23]:
lgb.save_model(file_name='lgb-124-9418-gkf-coup.pkl')
pd.DataFrame({"id": train_ids, "redemption_status": lgb_oof}).to_csv('lgb-124-9418-gkf-coup-oof.csv', index=False)
pd.DataFrame({"id": test_ids, "redemption_status": lgb_preds}).to_csv('lgb-124-9418-gkf-coup-test.csv', index=False)

'lgb-124-9418-gkf-coup.pkl'

XGBoost GroupKFold on CampaignID

In [24]:
params={
    'colsample_bytree': 0.8,
    'gamma': 0.9,
    'learning_rate': 0.1,
    'max_depth': 4,
    'min_child_weight': 10.0,
    'n_estimators': 10000,
    'objective': 'binary:logistic',
    'subsample': 0.8
}

xgb = Estimator(XGBClassifier(**params), early_stopping_rounds=500, n_splits=5, random_state=100, 
    variance_penalty=1, verbose=100, eval_metric='auc', scoring_metric=roc_auc_score, 
    validation_scheme=FoldScheme.GroupKFold, cv_group_col=group_col
)

xgb_oof = xgb.fit_transform(train.values, y.values)
xgb.avg_cv_score

/Users/Arvind/.local/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-auc:0.80326
[100]	validation_0-auc:0.93293
[200]	validation_0-auc:0.93183
[300]	validation_0-auc:0.92924
[400]	validation_0-auc:0.92544
[500]	validation_0-auc:0.92529
[590]	validation_0-auc:0.92415
[0]	validation_0-auc:0.89698
[100]	validation_0-auc:0.95476
[200]	validation_0-auc:0.95000
[300]	validation_0-auc:0.94920
[400]	validation_0-auc:0.94895
[500]	validation_0-auc:0.94787
[590]	validation_0-auc:0.94812
[0]	validation_0-auc:0.75313
[100]	validation_0-auc:0.84579
[200]	validation_0-auc:0.84016
[300]	validation_0-auc:0.82580
[400]	validation_0-auc:0.82525
[500]	validation_0-auc:0.81965
[589]	validation_0-auc:0.82544
[0]	validation_0-auc:0.79988
[100]	validation_0-auc:0.88985
[200]	validation_0-auc:0.90263
[300]	validation_0-auc:0.90275
[400]	validation_0-auc:0.90290
[500]	validation_0-auc:0.89775
[600]	validation_0-auc:0.89295
[657]	validation_0-auc:0.89150
[0]	validation_0-auc:0.79763
[100]	validation_0-auc:0.88561
[200]	validation_0-auc:0.87276
[300]	validation_0

0.9080882312148777

In [25]:
xgb_preds = xgb.transform(test[train.columns].values)

/Users/Arvind/Downloads/train_AUpWtIz/custom_estimator.py:188: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.mean(np.column_stack((est.predict_proba(x)[:,1] for est in self.fitted_models)), axis=1)
/Users/Arvind/.local/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


In [26]:
xgb.save_model(file_name='xgb-124-9119-gkf-camp.pkl')
pd.DataFrame({"id": train_ids, "redemption_status": xgb_oof}).to_csv('xgb-124-9119-gkf-camp-oof.csv', index=False)
pd.DataFrame({"id": test_ids, "redemption_status": xgb_preds}).to_csv('xgb-124-9119-gkf-camp-test.csv', index=False)

'xgb-124-9119-gkf-camp.pkl'

LightGBM GroupKFold on CouponID

In [27]:
params={
    'colsample_bytree': 0.8,
    'gamma': 0.9,
    'learning_rate': 0.01,
    'max_depth': 4,
    'min_child_weight': 10.0,
    'n_estimators': 10000,
    'objective': 'binary:logistic',
    'subsample': 0.8
}

xgb = Estimator(XGBClassifier(**params), early_stopping_rounds=500, n_splits=5, random_state=100, 
    variance_penalty=1, verbose=100, eval_metric='auc', scoring_metric=roc_auc_score, 
    validation_scheme=FoldScheme.GroupKFold, cv_group_col=group_col_coupon
)

xgb_oof = xgb.fit_transform(train.values, y.values)
xgb.avg_cv_score

/Users/Arvind/.local/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-auc:0.82378
[100]	validation_0-auc:0.91195
[200]	validation_0-auc:0.91792
[300]	validation_0-auc:0.93037
[400]	validation_0-auc:0.93592
[500]	validation_0-auc:0.93981
[600]	validation_0-auc:0.94587
[700]	validation_0-auc:0.94812
[800]	validation_0-auc:0.95024
[900]	validation_0-auc:0.95185
[1000]	validation_0-auc:0.95321
[1100]	validation_0-auc:0.95364
[1200]	validation_0-auc:0.95445
[1300]	validation_0-auc:0.95470
[1400]	validation_0-auc:0.95490
[1500]	validation_0-auc:0.95523
[1600]	validation_0-auc:0.95549
[1700]	validation_0-auc:0.95533
[1800]	validation_0-auc:0.95539
[1900]	validation_0-auc:0.95527
[2000]	validation_0-auc:0.95566
[2100]	validation_0-auc:0.95550
[2200]	validation_0-auc:0.95533
[2300]	validation_0-auc:0.95582
[2400]	validation_0-auc:0.95550
[2500]	validation_0-auc:0.95584
[2600]	validation_0-auc:0.95548
[2700]	validation_0-auc:0.95571
[2800]	validation_0-auc:0.95591
[2900]	validation_0-auc:0.95600
[3000]	validation_0-auc:0.95596
[3100]	validation_0-

0.9511478907294812

In [28]:
xgb_preds = xgb.transform(test[train.columns].values)

/Users/Arvind/Downloads/train_AUpWtIz/custom_estimator.py:188: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.mean(np.column_stack((est.predict_proba(x)[:,1] for est in self.fitted_models)), axis=1)
/Users/Arvind/.local/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


In [29]:
xgb.save_model(file_name='xgb-124-9514-gkf-coup.pkl')
pd.DataFrame({"id": train_ids, "redemption_status": xgb_oof}).to_csv('xgb-124-9514-gkf-coup-oof.csv', index=False)
pd.DataFrame({"id": test_ids, "redemption_status": xgb_preds}).to_csv('xgb-124-9514-gkf-coup-test.csv', index=False)

'xgb-124-9514-gkf-coup.pkl'